# CBC vs CTR: Changes in ciphertext
## Classic Crypto (FALL 2022-2023)

Write a Python program that will encrypt the image file “secret.bmp”, using AES-256-CBC and AES-256-CTR. 

You can choose a random key and IV/nonce for the encryption processes.

After encryption, introduce an error into the ciphertexts and decrypt the modified versions. Try, for example, changing some bytes right in the middle of the encrypted image data or anywhere after the first 54 bytes and setting them to 0.

After corrupting the data, decrypt the corrupted ciphertexts and view the two restored images.

How much of a difference did the edit make with CBC? How much of a difference did the edit make with CTR?
Padding should not be considered for this exercise.

In [ ]:

import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [ ]:
def encrypt_CBC(plain: bytes, key: bytes, iv: bytes):
    cipher = Cipher(algorithms.AES256(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    return encryptor.update(plain) + encryptor.finalize()


def encrypt_CTR(plain: bytes, key: bytes, nonce: bytes):
    cipher = Cipher(algorithms.AES256(key), modes.CTR(nonce))
    encryptor = cipher.encryptor()
    return encryptor.update(plain) + encryptor.finalize()


def corrupt(data: bytes):
    data_len = len(data)
    corrupted_data = list(data)
    corrupted_data[data_len//2-8:data_len//2+8] = [0]*16

    return bytes(corrupted_data)


def decrypt_CBC(ciphertext: bytes, key: bytes, iv: bytes):
    cipher = Cipher(algorithms.AES256(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    return decryptor.update(ciphertext) + decryptor.finalize()


def decrypt_CTR(ciphertext: bytes, key: bytes, nonce: bytes):
    cipher = Cipher(algorithms.AES256(key), modes.CTR(nonce))
    decryptor = cipher.decryptor()
    return decryptor.update(ciphertext) + decryptor.finalize()


![Processed Image](secret.bmp)

In [ ]:
def main():
    with open("secret.bmp", "rb") as secret:
        image = secret.read()

    header = image[:54]
    data = image[54:]

    key = secrets.token_bytes(32)
    iv = secrets.token_bytes(16)
    nonce = secrets.token_bytes(16)

    enc_CBC = encrypt_CBC(data, key, iv)
    enc_CTR = encrypt_CTR(data, key, nonce)

    corrupted_CBC = corrupt(enc_CBC)
    corrupted_CTR = corrupt(enc_CTR)

    dec_CBC = decrypt_CBC(corrupted_CBC, key, iv)
    dec_CTR = decrypt_CTR(corrupted_CTR, key, nonce)

    with open("secret-CBC.bmp", "wb") as secret:
        secret.write(header+dec_CBC)

    with open("secret-CTR.bmp", "wb") as secret:
        secret.write(header+dec_CTR)


if __name__ == "__main__":
    main()

![Processed Image](secret-CBC.bmp)

![Processed Image](secret-CTR.bmp)